<a href="https://colab.research.google.com/github/fatkhurizqirakhmansyah/learning_journal/blob/main/h8dsft_P0W3_Fatkhurizqi_Rakhmansyah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from google.colab import auth
from google.cloud import bigquery
from scipy import stats
import pandas as pd
import numpy as np

auth.authenticate_user()
print('Authenticated')

project_id = 'batch015' #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


Analisis SMART Thelook E commerce

S ( Spesific )
Meningkatkan jumlah user loyal dan mempertahankan jumlah user loyal yang sudah ada

M ( Measurable )
Sebanyak 40% kenaikan jumlah user loyal dari user loyal sebelumnya

A ( Attainable )
Memberikan diskon dalam pembelian dengan harga tertentu dan membuat harga beli borongan lebih murah

R ( Relevant )
Melihat banyak nya user pada e commerce tersebut, costumer akan membicarakan produk yang mereka beli di e commerce TheLook ke teman mereka, dengan pertimbangan tersebut akan dirasa cukup untuk menaikkan 40% jumlah user loyal hingga akhir tahun 2024

T ( Timely )
Q4 2024

Cara menjawab query seperti di bawah, setiap pertanyaan 1 query sql

Poin penjabaran:

1. Berapa persen jumlah user yang retention dari Januari - Juni 2022 dari total keseluruhan user yang ada? (Asumsi definisi retention minimal melakukan transaksi 6 kali selama periode) Apa kesimpulan/insight yang bisa kamu berikan?

In [9]:
# Query menampilkan data user retention periode januari 2022 hingga juni 2022
# User retention adalah user yang jumlah_transaksinya lebih besar dari sama dengan 3 selama periode

df = client.query('''SELECT  extract(year from orders.created_at) as year, orders.user_id, users.first_name, users.last_name, count(orders.num_of_item) as jumlah_transaksi
FROM `bigquery-public-data.thelook_ecommerce.orders` as orders
INNER JOIN `bigquery-public-data.thelook_ecommerce.users` as users
      ON users.id = orders.user_id
where orders.created_at between '2022-01-01' and '2022-06-30'
GROUP BY year, orders.user_id,users.first_name, users.last_name
having jumlah_transaksi >= 3
ORDER BY user_id, count(orders.num_of_item) asc''').to_dataframe()

df

,year,user_id,first_name,last_name,jumlah_transaksi
0,2022,64,Elizabeth,Mendoza,4
1,2022,441,Debra,Ortega,3
2,2022,556,Daniel,Schroeder,3
3,2022,620,Zachary,Shelton,3
4,2022,718,Reginald,Rocha,3
...,...,...,...,...,...
1030,2022,99864,Daniel,Heath,3
1031,2022,99891,Karen,Bennett,3
1032,2022,99903,Brandon,Brown,3
1033,2022,99931,Michael,Cowan,3


In [10]:
# Query untuk keseluruhan isi tabel bulan januari 2022 hingga juni 2022

df1 = client.query('''SELECT  extract(year from orders.created_at) as year, orders.user_id, users.first_name, users.last_name, count(orders.num_of_item) as jumlah_transaksi
FROM `bigquery-public-data.thelook_ecommerce.orders` as orders
INNER JOIN `bigquery-public-data.thelook_ecommerce.users` as users
      ON users.id = orders.user_id
where orders.created_at between '2022-01-01' and '2022-06-30'
GROUP BY year, orders.user_id,users.first_name, users.last_name
ORDER BY user_id, count(orders.num_of_item) asc''').to_dataframe()

df1

,year,user_id,first_name,last_name,jumlah_transaksi
0,2022,4,Larry,Ayala,1
1,2022,8,Michael,Wilkinson,1
2,2022,9,Kelly,Adams,1
3,2022,10,Thomas,Sanders,1
4,2022,14,Susan,Henry,1
...,...,...,...,...,...
30621,2022,99977,Karen,Clayton,3
30622,2022,99979,Cassandra,Jones,1
30623,2022,99983,John,Hernandez,1
30624,2022,99985,Emily,Johnson,1


In [16]:
# menghitung jumlah persentase user retention berdasarkan banyaknya baris tabel user retention
# dibagi dengan banyaknya total keseluruhan baris pada penjualan bulan januari hingga juni 2022

jumlah_persentase_retention = 1035 / 30626 * 100
jumlah_persentase_retention

3.379481486318814

**Dari hasil diatas persentase user retention TheLook E commerce sebesar 3.38%**, melihat kecil nya nilai user retention pada e commerce thelook, maka perlu dilakukan promosi agar orang lebih tertarik beli pada e commerce thelook dibanding e commerce lain, contoh nya dengan mengadakan diskon pembelian, memberikan perhatian lebih kepada user retention, sehingga user yang melakukan pembelian kurang dari sama dengan 3 kali akan lebih tertarik untuk membeli lebih dari produk yang dijual oleh thelook e commerce, memaksimalkan proses pengiriman barang, menambah distribusi di kota yang memiliki banyak costumer, mengajak kembali costumer yang telah pergi, mengajak pelanggan untuk membuat akun, meningkatkan komunikasi secara personal, memiliki banyak pilihan produk, melakukan pengenalan mengenai produk anda kepada pelanggan baru, memiliki layanan pelanggan yang baik, melakukan kegiatan menawarkan produk dengan angka yang lebih tinggi atau menawarkan pelanggan produk lain yang berkaitan dengan produk awal yang mereka beli atau cari

Poin Penjabaran :

2. Berikan informasi rata-rata, minimum, dan maksimum jumlah transaksi yang dilakukan oleh user loyal? (Tampilkan hanya dalam satu dataframe dan berdasarkan satu query select SQL) Informasi apa yang bisa kamu berikan?

In [48]:
# Query Min MAX Average User loyal

df2 = client.query('''select order_items.user_id as user_id, order_items.status, min(orders.num_of_item) as min_transaksi, max(orders.num_of_item) as max_transaksi, avg(orders.num_of_item) as rata_rata_transaksi, sum(orders.num_of_item) as jumlah_transaksi
from `bigquery-public-data.thelook_ecommerce.order_items` as order_items
left join `bigquery-public-data.thelook_ecommerce.orders` as orders
on order_items.user_id = orders.user_id
where order_items.status = 'Shipped'
GROUP BY order_items.user_id, order_items.status
having jumlah_transaksi >=6
ORDER BY user_id asc''').to_dataframe()

df2

,user_id,status,min_transaksi,max_transaksi,rata_rata_transaksi,jumlah_transaksi
0,9,Shipped,1,2,1.333333,8
1,17,Shipped,1,3,1.666667,10
2,25,Shipped,1,3,2.000000,12
3,26,Shipped,3,3,3.000000,9
4,33,Shipped,1,4,2.500000,20
...,...,...,...,...,...,...
9669,99944,Shipped,1,3,1.500000,24
9670,99945,Shipped,1,4,2.500000,20
9671,99952,Shipped,3,3,3.000000,9
9672,99991,Shipped,1,3,2.000000,12


**User Loyal disini saya Definisikan adalah user yang melakukan pembelian minimal sebanyak 6 kali**, mengapa disebut user loyal, karena user tersebut mau mengeluarkan dana lebih untuk membeli produk thelook, melihat banyaknya user loyal thelook akan lebih mudah untuk menaikkan jumlah user loyal dikarenakan thelook sudah mempunyai sistem yang bagus sehingga terdapat 9674 user loyal pada e commerce thelook, diharapkan langkah selanjutnya adalah menjaga user loyal yang sudah ada dan menambah user loyal sebanyak 40% hingga akhir tahun 2024 bisa dilakukan cara pengenalan pada pelanggan baru mengenai barang yang di jual oleh e commerce the look

Poin Penjabaran :

3. Berikan informasi rata-rata, minimum, dan maksimum uang yang dihabiskan oleh user loyal? (Tampilkan hanya dalam satu dataframe dan berdasarkan satu query select SQL) Informasi apa yang bisa kamu berikan?

In [50]:
# Query jumlah uang yang di keluarkan user loyal

df3 = client.query('''select order_items.user_id as user_id, order_items.status, min(order_items.sale_price) as min_dollar, max(order_items.sale_price) as max_dollar, avg(order_items.sale_price) as rata_rata_dollar, sum(order_items.sale_price) as total_dollar, sum(orders.num_of_item) as jumlah_transaksi
from `bigquery-public-data.thelook_ecommerce.order_items` as order_items
left join `bigquery-public-data.thelook_ecommerce.orders` as orders
on order_items.user_id = orders.user_id
where order_items.status = 'Shipped'
GROUP BY order_items.user_id, order_items.status
having jumlah_transaksi >= 6
ORDER BY user_id asc''').to_dataframe()

df3

,user_id,status,min_dollar,max_dollar,rata_rata_dollar,total_dollar,jumlah_transaksi
0,9,Shipped,20.000000,57.060001,38.530001,231.180004,8
1,17,Shipped,34.980000,154.000000,94.490000,566.939999,10
2,25,Shipped,10.000000,36.000000,23.663333,141.980000,12
3,26,Shipped,31.200001,52.950001,39.706668,119.120003,9
4,33,Shipped,19.000000,274.989990,93.394998,747.159981,20
...,...,...,...,...,...,...,...
9669,99944,Shipped,13.500000,139.990005,75.122501,1201.960022,24
9670,99945,Shipped,24.500000,134.000000,65.710000,525.680000,20
9671,99952,Shipped,9.660000,32.080002,20.576667,61.730001,9
9672,99991,Shipped,24.530001,49.500000,34.010000,204.060001,12


Dengan melihat total dollar yang telah dihabiskan oleh user loyal yang berada di kisaran 50 dollar ke atas dengan jumlah pembelian minimal 6 produk , maka rata rata user loyal minimal menghabiskan 8 dollar untuk belanja di thelook, artinya harga produk di e commerce thelook terjangkau untuk semua kalangan, hanya perlu di perbanyak target pasar baru untuk e commerce thelook dan coba mengaplikasikan kepada user loyal untuk membeli barang yang memiliki harga lebih tinggi, atau mengajak pelanggan untuk membeli barang lain di thelook yang berkaitan dengan produk pembelian awal mereka

Poin Penjabaran :

4. Berasal dari sumber trafik mana saja kebanyakan user loyal akses platform The Look? Informasi apa yang bisa kamu berikan?

In [52]:
# Query mencari sumber trafik user loyal

df4 = client.query('''select orders.user_id as user_id, orders.status, users.first_name, users.last_name, users.traffic_source, sum(orders.num_of_item) as jumlah_transaksi
from `bigquery-public-data.thelook_ecommerce.orders` as orders
left join `bigquery-public-data.thelook_ecommerce.users` as users
on orders.user_id = users.id
where orders.status = 'Shipped'
GROUP BY user_id, orders.status, users.traffic_source, users.first_name, users.last_name, users.traffic_source
having jumlah_transaksi >= 6
ORDER BY user_id asc''').to_dataframe()

df4

,user_id,status,first_name,last_name,traffic_source,jumlah_transaksi
0,149,Shipped,Holly,West,Search,6
1,622,Shipped,Phillip,Thompson,Search,7
2,628,Shipped,Anna,Sellers,Organic,6
3,1022,Shipped,Michael,Allen,Facebook,7
4,1451,Shipped,Michael,Green,Facebook,8
...,...,...,...,...,...,...
238,97784,Shipped,Christina,Hall,Search,6
239,98479,Shipped,Kelly,Lowe,Organic,6
240,98578,Shipped,Barbara,Garrison,Facebook,6
241,99688,Shipped,Erin,Reed,Search,8


In [54]:
# Mencari traffic source terbanyak

df4['traffic_source'].value_counts()

Search      172
Organic      36
Facebook     20
Email         9
Display       6
Name: traffic_source, dtype: int64

Dilihat dari data diatas, Traffic Source terbanyak di miliki oleh search dan organic search, yang artinya user loyal kebanyakan sudah mengetahui keberadaan situs thelook, mereka hanya tinggal mengetik alamat website thelook untuk melakukan transaksi atau hanya melihat produk di website thelook, hanya 35 user loyal yang mengakses thelook via social media, email and advertising on other website

Poin Penjabaran :

5. Apakah ada hubungan antara sumber trafik dengan loyal atau tidaknya seorang user? (Gunakan chi-square test untuk menguji hubungan itu) Hint: Output tabel query hanya ada dua kolom yaitu kolom 'trafic source' dan 'loyal' (kolom 'loyal' berisikan ya dan tidak saja

In [63]:
# Membuat dataframe baru untuk perhitungan chi-square test

data = {'Traffic_Source': ['Search', 'Search', 'Organic', 'Facebook', 'Search', 'Email', 'Display','Facebook','Search','Facebook',"Organic",'Organic','Search','Facebook','Search'],
        'Loyal': ['Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', "Yes", 'Yes', 'No','Yes','No','No']}

df5 = pd.DataFrame(data)
df5

,Traffic_Source,Loyal
0,Search,Yes
1,Search,No
2,Organic,Yes
3,Facebook,No
4,Search,Yes
5,Email,Yes
6,Display,Yes
7,Facebook,Yes
8,Search,Yes
9,Facebook,Yes


In [64]:
# Memuat Cross tab untuk perhitungan chi square test

contingency_table=pd.crosstab(df5["Traffic_Source"], df5["Loyal"])
contingency_table

Loyal,No,Yes
Traffic_Source,,
Display,0,1
Email,0,1
Facebook,2,2
Organic,1,2
Search,2,4


In [65]:
# Menentukan apakah ada hubungan traffic source dengan loyal atau tidak nya user

stat, p, dof, expected = stats.chi2_contingency(contingency_table)
print('p=%.3f' % (p))
if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

p=0.827
Probably independent


Jawaban Probably Indepent mengartikan bahwa tidak adanya hubungan antara traffic_source yang di gunakan dengan loyal atau tidaknya seorang user

POIN ANALISIS

Berikan kesimpulan dari laporan/informasi yang dibuat berdasarkan problem statement/poin penjabaran menggunakan bahasa awam! (boleh dalam beberapa paragraf)

Dengan dimiliki nya user loyal yang banyak, dan user yang membeli barang berulang kali minimal sebanyak 3 kali pada periode januari 2022 hingga juni 2022 berjumlah 3,38% maka kemungkinan untuk dipenuhi nya tujuan dari analisis smart yaitu meningkat kan jumlah user loyal sebanyak 40% di akhir tahun 2024 akan mungkin tercapai, jika e commerce thelook mampu menjaga user yang sudah ada dan mendapatkan pelanggan baru yang kemungkinan akan menjadi user loyal juga, dengan meningkatkan pelayanan dan distribusi produk kepada user, lebih banyak mengenalkan penggunaan produk agar masyarakat awam tau akan manfaat produk yang dijual di e commerce the look

POIN PERTANYAAN

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

Dapat tercapai, jika platform thelook meningkatkan pelayanan kepada user yang telah ada dan memberikan loyalty program kepada setiap user yang menggunakan platform thelook, contoh nya mendapatkan satu buah produk gratis setelah pembelian minimal 5 kali di platform thelook, dapat pula memberikan voucher diskon untuk pembelian dengan harga tertentu, atau program membeli 3 barang gratis satu barang dalam periode tertentu dan mengajak pembeli untuk melihat barang yang lebih tinggi nilai nya, atau membuat pelanggan membeli produk lainnya yang berkaitan dengan produk awal yang pelanggan beli

2. Apa yang dimaksud dengan customer/user retention?

Customer / User Retention adalah proses mempertahankan pelanggan yang sudah ada dan menarik pelanggan baru untuk terus membeli produk atau menggunakan jasa bisnismu lagi

3. Jika platform The Look membuat fitur tambahan, apakah kamu akan merekomendasikan/memprioritaskan kepada user loyal? apa pertimbanganmu?

Ya saya akan merekomendasikan kepada user loyal, hal ini dikarenakan user tersebut secara tidak langsung akan menaikkan traffic seiring dengan sering nya user tersebut berbelanja di e commerce kita, akan tetapi harus dilihat dulu apakah si pembeli tersebut nyaman atau tidak dengan fitur tambahan yang akan dibuat, jangan membuat fitur tambahan yang tidak ada manfaat nya bagi para pengguna, misal fitur tersebut membuat pengguna harus menekan link link yang dapat berpotensi terjadinya penipuan atau scam yang mengakibatkan kerugian pada pengguna